**<font color=#FF0000>Fraud </font> Package Installation Analysis**

## 🔍 Fraud Detection in Python Package Installations Using BigQuery Logs

In this notebook, we analyze command-line logs to detect suspicious `pip install` activities and identify potential security risks such as:

1. **Unauthorized or malicious package installations**  
   Using known malicious package data (e.g., from JFrog Artifactory threat reports), we flagged `pip install` commands that install packages associated with malware or typosquatting attacks.

2. **Hardcoded secrets in command lines**  
   We detect hardcoded passwords and credentials accidentally exposed in CLI arguments using regex patterns.

🔑 For both types of threats, we extract:
- The original `insert_id` values for easy traceability back to the logs.
- Metadata such as project ID, timestamp, and command text.

In [ ]:
!pip install --upgrade google-cloud-bigquery pandas

  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.31.0
    Uninstalling google-cloud-bigquery-3.31.0:
      Successfully uninstalled google-cloud-bigquery-3.31.0


In [ ]:
from google.cloud import bigquery
import pandas as pd

In [ ]:
# Create a client object
client = bigquery.Client()

In [ ]:
# Correct SQL with backticks
query = """
SELECT *
FROM `fraud-459711.fraud_dataset.fraud_data`
LIMIT 50
"""

# Run the query and get an iterator over the rows
query_job = client.query(query)

# Get results
results = query_job.result()  # Waits for job to complete

for row in results:
    print(row)


Row(('User: jovyan, Hostname: user-1, Input: \ncode = """ //\nconst UNIFIED_ORDER_URL = \'https://api.mch.weixin.qq.com/pay/unifiedorder\';\n//\nconst FIND_ORDER_URL = \'https://api.mch.weixin.qq.com/pay/orderquery\';\nprivate $appid = \'\'; //appid\nprivate $secret = \'dads\'; //appsecret\nprivate $mchid = \'\'; //\nprivate $key = \'\'; //\nprivate $sslcert_path = \'\'; //\nprivate $sslkey_path = \'\'; //\npublic function __construct($appid = \'\', $secret = \'\', $mchid = \'\', $key = \'\')\n{\nif (!empty($appid)) $this->appid = $appid;"""\nwith default_settings():\n    result = main.scan_adhoc_string(code)\n    print(result)', 'projects/db-dev-k0dy-pbawp-srh/logs/jn-logs', '', 'db-dev-k0dy-pbawp-srh', datetime.datetime(2025, 1, 28, 14, 11, 55, 635975, tzinfo=datetime.timezone.utc), '1p1p8t0flqk55n', 'kubernetes.pod.interaction', 'fe94fb9a-0b36-4a7d-a40c-1d8f23dbc21d'), {'user_interaction_awp': 0, 'logName': 1, 'user_combined': 2, 'project_id': 3, 'timestamp': 4, 'insertId': 5, 'acti

In [ ]:
from google.cloud import bigquery

# Create a client
client = bigquery.Client()

# Reference the table
table_ref = client.get_table("fraud-459711.fraud_dataset.fraud_data")

# Print all column names
column_names = [schema_field.name for schema_field in table_ref.schema]
print(column_names)

for field in table_ref.schema:
    print(f"{field.name} ({field.field_type})")

['user_interaction_awp', 'logName', 'user_combined', 'project_id', 'timestamp', 'insertId', 'action', 'dsId']
user_interaction_awp (STRING)
logName (STRING)
user_combined (STRING)
project_id (STRING)
timestamp (TIMESTAMP)
insertId (STRING)
action (STRING)
dsId (STRING)


In [ ]:
from google.cloud import bigquery

# Create BigQuery client
client = bigquery.Client()

# SQL: Suspicious pip installs with file extensions like zip, gzip, tar.gz, etc.
suspicious_query = """
SELECT
  insertId,
FROM
  `fraud-459711.fraud_dataset.fraud_data`
WHERE
  user_interaction_awp IS NOT NULL
  AND REGEXP_CONTAINS(LOWER(CAST(user_interaction_awp AS STRING)), r'pip install')
  AND REGEXP_CONTAINS(LOWER(CAST(user_interaction_awp AS STRING)), r'\\.(zip|gzip|tar\\.gz|whl|bz2)')
"""

# Run the query
query_job = client.query(suspicious_query)
results = query_job.result()

# Print each suspicious record
print("🔍 Suspicious pip installs with archive formats:\n")
for row in results:
    print(f"insertId: {row.insertId}")

🔍 Suspicious pip installs with archive formats:

insertId: vn3lhgfkjyn7t
insertId: 12bq7bgf1ybo2h
insertId: 18edk33fm81dcb
insertId: ofrdb7fjjx4ta
insertId: 1xdggbodnzu7
insertId: gowdumf1zqop8
insertId: 16rwyyxf18ndeg
insertId: w84whrfanhjd3
insertId: qxbo5gflhqotk
insertId: 185pr94eh7tb0
insertId: xwp2gje2tyke
insertId: ofrdb7fjjx4ta
insertId: 1bqs4bsf1641o1
insertId: 1ea28jkf150c0p
insertId: vn3lhgfkjyn7t
insertId: 1xdggbodnzu7
insertId: 12bq7bgf1ybo2h
insertId: gowdumf1zqop8
insertId: ritjewfkif1l6
insertId: lnebefjjuyks
insertId: 18edk33fm81dcb


In [ ]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client()

# SQL query to detect hardcoded passwords
query = """
SELECT
  insertId
FROM
  `fraud-459711.fraud_dataset.fraud_data`
WHERE
  user_interaction_awp IS NOT NULL
  AND REGEXP_CONTAINS(
    LOWER(CAST(user_interaction_awp AS STRING)),
    r'(password\\s*=\\s*["\\\']?.+?["\\\']?|--password\\s+\\S+|pwd\\s*=\\s*["\\\']?.+?["\\\']?|["\\\']password["\\\']\\s*:\\s*["\\\']?.+?["\\\'])'
  )
"""

# Execute the query
query_job = client.query(query)
results = query_job.result()

# Print each suspicious record
print("🔍 Suspicious pip installs with hardcoded passwords:\n")
for row in results:
    print(f"insertId: {row.insertId}")

🔍 Suspicious pip installs with hardcoded passwords:

insertId: nddioufl2qu1u
insertId: 1c120y5f1aviam
insertId: efjg9bf67e1of
insertId: 1gr4wqhf21m97f
insertId: 1havxnlf1zlma1
insertId: 1fxaiqjf232qkj
insertId: 1aco6l4f20easu
insertId: h8do8lf1jckl4
insertId: aaj7k1f1y8257
insertId: 18oa3h7f20o33z
insertId: 7rtflif6j4z69
insertId: kkn0dsfjqh61c
insertId: s2a9bkfjj9uxx
insertId: 16r3jigf3kl415
insertId: rrsnuifkm63ta
insertId: ub058of4hr6f5
insertId: 1q5jogdf8z2xgj
insertId: 153uur4fk1b1q8
insertId: 3n2l41fjr2s62
insertId: 1kn9olee1tojb
insertId: 1b7p025f3vl6uy
insertId: egtgv6f4gs7ri
insertId: 1ei1hcxf3k5k5e
insertId: 1plfe3e1dsjk
insertId: alhiykf1fl6yu
insertId: iwztjgf20005r
insertId: x1rarzf20p25z
insertId: 7igjn1f15hk0r
insertId: 1kn9olee1tojb
insertId: x1rarzf20p25z
insertId: 7rtflif6j4z69
insertId: efjg9bf67e1of
insertId: h8do8lf1jckl4
insertId: aaj7k1f1y8257
insertId: 1plfe3e1dsjk
insertId: 153uur4fk1b1q8
insertId: b4coyflioqfo
insertId: nddioufl2qu1u
insertId: alhiykf1fl6yu
in

In [ ]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client()

# SQL query to extract insertIds where password=... is detected
query = """
SELECT
  insertId
FROM
  `fraud-459711.fraud_dataset.fraud_data`
WHERE
  user_interaction_awp IS NOT NULL
  AND REGEXP_CONTAINS(
    LOWER(CAST(user_interaction_awp AS STRING)),
    r'password\\s*=\\s*["' || "'" || r']?.+?["' || "'" || r']?'
  )
"""

# Execute the query
query_job = client.query(query)
results = query_job.result()

# Display results
print("🆔 Insert IDs with hardcoded passwords:\n")
for row in results:
    print(f"insertId: {row.insertId}")

🆔 Insert IDs with hardcoded passwords:

insertId: nddioufl2qu1u
insertId: 1c120y5f1aviam
insertId: 153uur4fk1b1q8
insertId: 3n2l41fjr2s62
insertId: 7igjn1f15hk0r
insertId: 153uur4fk1b1q8
insertId: nddioufl2qu1u
insertId: 1c120y5f1aviam
insertId: eq93jzflzpsrh
insertId: 3n2l41fjr2s62
insertId: 7igjn1f15hk0r


In [ ]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client()

# SQL query to extract hardcoded passwords (just password=... pattern)
query = """
SELECT
  REGEXP_EXTRACT(
    LOWER(CAST(user_interaction_awp AS STRING)),
    r'password\\s*=\\s*["' || "'" || r']?([^"' || "'" || r']+)'
  ) AS potential_password
FROM
  `fraud-459711.fraud_dataset.fraud_data`
WHERE
  user_interaction_awp IS NOT NULL
  AND REGEXP_CONTAINS(
    LOWER(CAST(user_interaction_awp AS STRING)),
    r'password\\s*=\\s*["' || "'" || r']?.+?["' || "'" || r']?'
  )
"""

# Execute the query
query_job = client.query(query)
results = query_job.result()

# Display results
print("🔐 Hardcoded passwords detected:\n")
for row in results:
    print(f"password: {row.potential_password}")

🔐 Hardcoded passwords detected:

password: 3423 fsd fsd 
password: dsa
password: fasdfs
password: fasdfs
                }
            }
        }
    }
}
password: os.environ.get(
password: fasdfs
password: 3423 fsd fsd 
password: dsa
password:  
password: fasdfs
                }
            }
        }
    }
}
password: os.environ.get(


In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

query = """
WITH extracted_packages AS (
  SELECT
    insertId,
    user_interaction_awp,
    REGEXP_EXTRACT(user_interaction_awp, r'pip\\s+install\\s+(\\S+)') AS package
  FROM
    `fraud-459711.fraud_dataset.fraud_data`
  WHERE
    user_interaction_awp IS NOT NULL
    AND REGEXP_CONTAINS(user_interaction_awp, r'pip\\s+install\\s+\\S+')
)

SELECT
  e.insertId,
  e.user_interaction_awp,
  e.package
FROM
  extracted_packages e
JOIN
  `fraud-459711.fraud_dataset.blocklist_packages` b
ON
  LOWER(e.package) = LOWER(b.package_name)
"""

query_job = client.query(query)
results = list(query_job.result())  # Materialize results to list

if not results:
    print("✅ No installs of blocklisted packages were detected.")
else:
    print("🚩 Detected installs of blocklisted packages:\n")
    for row in results:
        print(f"insertId: {row.insertId}, package: {row.package}, command: {row.user_interaction_awp}")

✅ No installs of blocklisted packages were detected.
